# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 13.6 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [7]:
fgsm_path = '/content/drive/MyDrive/Dataset/insdn/adv/24_feature/fgm/fgm_mlp_bin_feature.csv'
#fgsm_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/fgsm/adv_fgsm_target.csv'

In [8]:
fgsm_df = pd.read_csv(fgsm_path)

In [9]:
fgsm_df.columns = feature_list
fgsm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

In [ ]:
#fgsm_label = pd.read_csv(fgsm_label_path)
#fgsm_label.info()

# Load DL-based IDS model

In [10]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [12]:
fgsm_1000 = fgsm_df.head(1000)

In [13]:
shap_values = explainer.shap_values(fgsm_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
data_df = pd.DataFrame(shap_values[0])

In [16]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000000,0.00000,0.005206,0.011182,0.007525,0.0,0.004171,-0.008295,0.000000,-0.011595,...,-0.011714,0.0,0.000000,0.000000,0.018712,-0.008680,0.013444,-0.009724,0.002248,0.003200
1,0.000000,0.00000,0.000000,-0.009904,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.041337,0.000000,-0.027067,-0.064104
2,0.013691,0.00000,0.000000,0.005674,0.004469,0.0,0.017167,0.000000,-0.003568,0.000000,...,-0.007757,0.0,0.000000,-0.007466,-0.009319,0.000000,-0.047403,0.000000,-0.036143,-0.041568
3,-0.004843,0.00373,0.004319,0.000000,-0.007645,0.0,0.009867,0.000000,0.003960,0.004352,...,0.000000,0.0,0.000000,0.000000,-0.007572,0.006459,-0.053761,-0.012276,-0.037872,-0.052935
4,-0.068196,0.00000,0.003978,-0.010479,-0.005654,0.0,-0.005390,-0.005511,0.000000,0.003899,...,-0.004918,0.0,0.098404,-0.218128,0.070281,0.008154,-0.183210,-0.007772,-0.171425,-0.141032


In [29]:
data_df.to_csv('/content/drive/MyDrive/1000_samples_bin_[0].csv', index=False)

In [17]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.00000,0.005206,0.011182,0.007525,0.0,0.004171,-0.008295,0.000000,-0.011595,...,-0.011714,0.0,0.000000,0.000000,0.018712,-0.008680,0.013444,-0.009724,0.002248,0.003200
1,0.000000,0.00000,0.000000,-0.009904,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.041337,0.000000,-0.027067,-0.064104
2,0.013691,0.00000,0.000000,0.005674,0.004469,0.0,0.017167,0.000000,-0.003568,0.000000,...,-0.007757,0.0,0.000000,-0.007466,-0.009319,0.000000,-0.047403,0.000000,-0.036143,-0.041568
3,-0.004843,0.00373,0.004319,0.000000,-0.007645,0.0,0.009867,0.000000,0.003960,0.004352,...,0.000000,0.0,0.000000,0.000000,-0.007572,0.006459,-0.053761,-0.012276,-0.037872,-0.052935
4,-0.068196,0.00000,0.003978,-0.010479,-0.005654,0.0,-0.005390,-0.005511,0.000000,0.003899,...,-0.004918,0.0,0.098404,-0.218128,0.070281,0.008154,-0.183210,-0.007772,-0.171425,-0.141032


In [18]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [19]:
print(data_dict)

{'Adv sample 0': ['Bwd IAT Min', 'Bwd IAT Max', 'Bwd IAT Mean', 'Init Bwd Win Byts', 'Fwd Pkts/b Avg', 'Pkt Len Mean', 'Pkt Len Min', 'Down/Up Ratio', 'Fwd IAT Mean', 'Pkt Len Std'], 'Adv sample 1': ['SYN Flag Cnt', 'Bwd IAT Mean', 'Fwd IAT Min', 'Bwd IAT Max', 'Init Bwd Win Byts', 'Fwd Byts/b Avg', 'Bwd Pkts/s', 'Bwd IAT Tot', 'Fwd IAT Mean', 'Bwd Seg Size Avg'], 'Adv sample 2': ['SYN Flag Cnt', 'Bwd IAT Mean', 'Fwd IAT Min', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd Pkts/b Avg', 'Fwd Byts/b Avg', 'Bwd Pkts/s', 'Fwd Seg Size Avg', 'Fwd Pkt Len Min'], 'Adv sample 3': ['SYN Flag Cnt', 'Bwd IAT Mean', 'Fwd IAT Min', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Fwd Byts/b Avg', 'Pkt Len Min', 'Flow Byts/s', 'URG Flag Cnt', 'Down/Up Ratio'], 'Adv sample 4': ['ACK Flag Cnt', 'Active Mean', 'Active Max', 'Init Bwd Win Byts', 'Flow IAT Std', 'Flow Pkts/s', 'Pkt Len Min', 'Flow IAT Min', 'Flow IAT Mean', 'Fwd IAT Max'], 'Adv sample 5': ['Bwd IAT Mean', 'Bwd IAT Min', 'Fwd Pkts/b Avg', 'Init Bwd Win Byt

In [20]:
len(shap_values[0])

1000

In [21]:
len(data_dict)

1000

# Detection phase

In [22]:
import os
import pandas as pd

In [23]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/InSDN/MLP_top24_InSDN_bin.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Init Bwd Win Byts,684
1,Idle Max,604
2,Bwd Pkt Len Max,571
3,Idle Mean,535
4,Idle Min,535
5,Bwd Pkt Len Mean,462
6,Bwd Pkt Len Std,444
7,Pkt Len Min,383
8,Tot Fwd Pkts,364
9,Bwd Seg Size Avg,358


## Detection rate (top 24) = 1000/1000

In [24]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std']


In [25]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:1000


## Detection rate (top 38) = 895/1000

In [26]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std', 'Fwd Pkts/s', 'Active Min', 'Fwd IAT Std', 'Flow IAT Mean', 'Fwd IAT Mean', 'Bwd Header Len', 'Pkt Len Var', 'Active Mean', 'Fwd IAT Tot', 'Active Max', 'Idle Std', 'Active Std', 'Flow IAT Std', 'Subflow Bwd Pkts']


In [27]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:1000
